# Lösen eines FEM Problems mit UG4

Wir lösen hier 
$$-\triangle u = f $$
mit Dirichlet-Randwerten.

Wir verwenden hier einen entprechenden Jupyter-Kernel, der LUA unterstützt. 

## Initialisierung 

In [1]:
InitUG(2, AlgebraType("CPU", 1));  -- Initialize world dimension dim=2 and default algebra type
ug_load_script("ug_util.lua")           -- Load utility scripts (e.g. from from ugcore/scripts)
ug_load_script("util/refinement_util.lua")

* Initializing: paths... done, bridge... done, plugins... fail                 *


## Problemdefinition

Zur Problemdefinition definieren wir einige Variablen:

In [2]:
myGridName= "grids/unit_square_quad.ugx" --"grids/unit_square_tri.ugx",
myNumRefs= 2
mySubsets = {"Inner", "Boundary"}

Randbedingungen setzen wir über einen Callback:

In [3]:
-- Callback fuer Randbedingungen
function myDirichletBndCallback(x, y, t)
    if (y==1) then 	return true, 0.0 
    elseif (y==0) then  return true, math.sin(math.pi*1*x)
    else return false, 0.0 
    end
end

## Rechengebiet
- Einlesen aus einer Datei:

In [4]:
dom = Domain()
LoadDomain(dom, myGridName)

- Test: Sind alle Subsets vorhanden?

In [5]:
print(util.CheckSubsets(dom, mySubsets))

true


- Uniforme Verfeinerung

In [6]:
local refiner = GlobalDomainRefiner(dom)
for i=1,myNumRefs do
    write(i .. " ")
    refiner:refine()
end

1 2 

- Alternativ geht es auch ganz bequem in einer Zeile:

In [7]:
dom = util.CreateDomain(myGridName, myNumRefs, myRequiredSubsets)

Loading Domain grids/unit_square_quad.ugx ... done.
Performing integrity check on domain ... done.
Refining(2): 1 2 done.


## Ansatzraum
Erzeuge Ansatzraum mit stückweise linearen Funktionen (elementweise Polynome 1. Grades):

In [8]:
-- Setup for FEM approximation space.
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct("u", "Lagrange", 1)  -- Linear ansatz functions

-- More inits.
approxSpace:init_levels()
approxSpace:init_top_surface()
approxSpace:print_statistic()

| ---------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                  |
|  Algebra: Block 1 (divide by 1 for #Index)                                   |
|                                                                              |
|    GridLevel   |       Domain |     0: Inner |  1: Boundary                  |
| ---------------------------------------------------------------------------- |
| (lev,    0)    |            9 |            1 |            8 |
| (lev,    1)    |           25 |            9 |           16 |
| (lev,    2)    |           81 |           49 |           32 |
| (lev,    0, g) |            9 |            1 |            8 |
| (lev,    1, g) |           25 |            9 |           16 |
| (lev,    2, g) |           81 |           49 |           32 |
| (surf, top)    |           81 |           49 |           32 |
| ----------------------------------------------------------------

## Diskretisierung

Erzeuge Objekt für eine **Elementdiskretisierung** für die Konvektions-Diffusionsgleichung.

In [9]:
elemDisc = ConvectionDiffusion("u", "Inner", "fe")
elemDisc:set_diffusion(1.0)

-- Optional: Setze rechte Seite f
-- elemDisc:set_source(1.0)

Erzeuge Objekt für **Randbedingungen**:

In [10]:
dirichletBND = DirichletBoundary()
dirichletBND:add("myDirichletBndCallback", "u", "Boundary")

Füge beides zu einer Gebietsdiskretisierung hinzu:

In [11]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc)
domainDisc:add(dirichletBND)

## Konfiguration eines iterativen Lösers

Ein Mehrgitterverfahren hat lediglich lineare Komplexität

In [12]:
-- set up solver (using 'util/solver_util.lua')
local solverDesc = {
    type = "bicgstab",
    precond = {
        type = "gmg",
        approxSpace = approxSpace,
        smoother = "sgs",
        baseSolver = "lu"
    }
}
solver = util.solver.CreateSolver(solverDesc)

## Assembliere und löse LGS

In [13]:
A = AssembledLinearOperator(domainDisc)
x = GridFunction(approxSpace)
b = GridFunction(approxSpace)
x:set(0.0)


domainDisc:assemble_linear(A, b)
domainDisc:adjust_solution(x)

solver:init(A, x)
solver:apply(x, b)


   % %%%%%%%%             BiCGStab              %%%%%%%%%%%
   % %%%%%%%%   (Precond: Geometric MultiGrid)  %%%%%%%%%%%
   %   Iter      Defect         Rate 
   %    0:    1.907058e+00      -------
   %    1:    1.485603e-02    7.790028e-03
   %    2:    1.754873e-04    1.181253e-02
   %    3:    3.511606e-06    2.001060e-02
   %    4:    9.332521e-09    2.657622e-03
   % Relative reduction 1.000000e-06 reached after 4 steps.
   % Average reduction over 4 steps: 8.363899e-03
   % %%%%%  Iteration converged  %%%%%



### Ausgabe als *.vtu bzw. *.vec-Datei
a) Für die Lösung

In [14]:
local solFileName = "x_solution"
WriteGridFunctionToVTK(x, solFileName)
SaveVectorForConnectionViewer(x, solFileName .. ".vec")

b) Für die rechte Seite $b$ und Matrix $A$

In [15]:
local matFileName = "A_matrix.mat"
SaveMatrixForConnectionViewer(x, A, matFileName)

local rhsFileName = "b_rhs"
SaveVectorForConnectionViewer(b, rhsFileName.. ".vec")